# NREL - Offshore WA OR - turbine: 25915

## Predicción: h = 12 / H = 24

## Model: Neural Network - Stack LSTM

In [1]:
%matplotlib inline
import os
import sys
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# import RenewAI
sys.path.append('/Users/tmorales/tmp/RNN-windPower/src')
from pre_process.split_ml_ts import dataframe_split

from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.externals import joblib
from sklearn.metrics.regression import mean_squared_error
from sklearn.metrics.regression import mean_absolute_error

In [2]:
exp_path = '/Users/tmorales/tmp/experiments/NREL/Offshore_WA_OR/turbine_25915/lag_5_h_12'
experiment_name = 'nn-stack-lstm'
directory = os.path.join(exp_path, experiment_name)
if not os.path.exists(directory):
    os.makedirs(directory)

## Load file

In [4]:
path = '/Users/tmorales/tmp/RNN-windPower/database/wind_farms/Offshore_WA_OR/Offshore_WA_OR_features'

In [5]:
os.chdir(path)
files = [file for file in glob.glob('Offshore_WA_OR_features_*.csv')]
files

['Offshore_WA_OR_features_h_01.csv',
 'Offshore_WA_OR_features_h_02.csv',
 'Offshore_WA_OR_features_h_03.csv',
 'Offshore_WA_OR_features_h_04.csv',
 'Offshore_WA_OR_features_h_05.csv',
 'Offshore_WA_OR_features_h_06.csv',
 'Offshore_WA_OR_features_h_07.csv',
 'Offshore_WA_OR_features_h_08.csv',
 'Offshore_WA_OR_features_h_09.csv',
 'Offshore_WA_OR_features_h_10.csv',
 'Offshore_WA_OR_features_h_11.csv',
 'Offshore_WA_OR_features_h_12.csv',
 'Offshore_WA_OR_features_h_13.csv',
 'Offshore_WA_OR_features_h_14.csv',
 'Offshore_WA_OR_features_h_15.csv',
 'Offshore_WA_OR_features_h_16.csv',
 'Offshore_WA_OR_features_h_17.csv',
 'Offshore_WA_OR_features_h_18.csv',
 'Offshore_WA_OR_features_h_19.csv',
 'Offshore_WA_OR_features_h_20.csv',
 'Offshore_WA_OR_features_h_21.csv',
 'Offshore_WA_OR_features_h_22.csv',
 'Offshore_WA_OR_features_h_23.csv',
 'Offshore_WA_OR_features_h_24.csv']

##  Paso 1: only one h-step

In [12]:
h11 = files[11]
h11

'Offshore_WA_OR_features_h_12.csv'

In [9]:
# create directory for each horizont (each horizon is a regressor)
#horizont = 12
#directory_by_horizont = os.path.join(directory, 
#                                     'K-nearest-regressor',
#                                     'model_saved_h={0:03d}'.format(horizont)
#                                    )
#if not os.path.exists(directory_by_horizont):
#    os.makedirs(directory_by_horizont)

In [13]:
# load file
df_h11 = pd.read_csv(h11, delimiter=';', index_col=0)
df_h11.head(4)

,f_0,f_1,f_2,f_3,f_4,f_5,target_h12
target_date,,,,,,,
2004-01-01 17:00:00,6.96,5.28,5.32,4.93,4.05,3.95,6.11
2004-01-01 18:00:00,5.28,5.32,4.93,4.05,3.95,4.19,6.42
2004-01-01 19:00:00,5.32,4.93,4.05,3.95,4.19,4.89,5.74
2004-01-01 20:00:00,4.93,4.05,3.95,4.19,4.89,5.16,4.52


In [14]:
# select the best model for h=1

# split the dataset in train, val and test
data = dataframe_split(df_h11)
train_set = data['train_set']
validation_set = data['validation_set']
test_set = data['test_set']


# features and target columns
features_columns = train_set.columns[1:-1]
target_column = train_set.columns[-1:]

# select the values of features and target columns.
features_train_set = train_set[features_columns]
target_train_set = train_set[target_column]
features_validation_set = validation_set[features_columns]
target_validation_set = validation_set[target_column]
features_test_set = test_set[features_columns]
target_test_set = test_set[target_column]

In [15]:
print('features columns: {0}'.format(features_columns))
print('target column: {0}'.format(target_column))

features columns: Index(['f_1', 'f_2', 'f_3', 'f_4', 'f_5'], dtype='object')
target column: Index(['target_h12'], dtype='object')


In [17]:
x_train = features_train_set.values; y_train = target_train_set.values
x_val = features_validation_set.values; y_val = target_validation_set.values

## Paso 2: Model